### Install

In [ ]:
%pip install -U langchain-community langgraph langchain-google-genai tavily-python langgraph-checkpoint-sqlite

In [ ]:
%pip install python-dotenv

In [2]:
import getpass
import os
from dotenv import load_dotenv

os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

### Create model

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)

### Project Info Agent
Extracts problem and make/model

In [ ]:
messages = [
    (
        "system",
        "You are a 'Project-info' agent that extracts a *problem* and *make/model* from a user's query. Return Unknown if inappropriate query."

    ),
    ("human", "How do I clean my MSI GS66 laptop's fan?"),

]

response = llm.invoke(messages)
response
response.content

### Project Info Agent


In [38]:
from langchain_core.prompts import ChatPromptTemplate

project_info_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a 'Project-info' agent that extracts a *problem* and" 
            "*make/model* from a user's query. Return Unknown if inappropriate"
            "query. Return in valid JSON format"
            
        ),
        ("human", "{input}"),
    ]

)


### Instructions Agent


In [ ]:
instruction_agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an 'Instruction-generator' agent that "
            "provides a list of step-by-step instructions to solve the"
            " given *problem* for the specified *make/model*. Provide it in JSON format with step1, step2, ... as the keys"
        ),
        ("human", "Problem: {problem}, Make/Model: {make_model}"),
    ]
)

### Chaining

In [57]:
# user_input = "How do I clean my MSI GS66 laptop's fan?"
# user_input = "What is the weather today"
user_input = "Where can I find the memory card location on my nintendo switch?"

project_info_chain = project_info_prompt | llm
project_info_response = project_info_chain.invoke({"input": user_input})

project_info_content = project_info_response.content

project_info_content = project_info_content.replace("```", "")
project_info_content = project_info_content.replace("json", "")
project_info = eval(project_info_content)
print(project_info_content)
print(project_info)


{
  "problem": "memory card location",
  "make/model": "nintendo switch"
}

{'problem': 'memory card location', 'make/model': 'nintendo switch'}


In [58]:
problem = project_info.get("problem", "Unknown")
make_model = project_info.get("make/model", "Unknown")

if problem != "Unknown" and make_model != "Unkown":
    instruction_chain = instruction_agent_prompt | llm
    instruction_response = instruction_chain.invoke({
        "problem": problem,
        "make_model": make_model
    })
    print(instruction_response.content)

# print(instruction_response)

Okay, I can provide instructions on how to locate the memory card slot on a Nintendo Switch.

**Instructions:**

1.  **Power Down:** Ensure your Nintendo Switch is completely powered off before attempting to locate or access the microSD card slot.

2.  **Locate the Kickstand:** On the back of the Nintendo Switch console, find the kickstand. It's a small, hinged flap that allows you to prop up the Switch on a flat surface.

3.  **Open the Kickstand:** Gently open the kickstand.

4.  **Find the microSD Card Slot:** Look for a small, rectangular slot located underneath the kickstand, near the hinge. It will be labeled. This is the microSD card slot.
